<a href="https://colab.research.google.com/github/Osondu-ifunanya/Invasive-species-mapping-using-remote-sensing-and-anomaly-detection/blob/main/Invasive%20species%20mapping%20using%20remote%20sensing%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Invasive Species Mapping using Remote Sensing and Anomaly Detection
(Synthetic Multispectral Satellite Data)

Workflow:
1. Generate synthetic multispectral imagery
2. Inject invasive species spectral anomaly
3. Compute vegetation index (NDVI)
4. Apply Isolation Forest for anomaly detection
5. Visualize invasive areas
6. Export anomaly map
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

np.random.seed(42)

# -------------------------------------------
# 1. Generate Synthetic Multispectral Image
# -------------------------------------------

# Image size
size = 200

# Simulated bands: Blue, Green, Red, NIR
blue = np.random.normal(0.1, 0.02, (size, size))
green = np.random.normal(0.15, 0.03, (size, size))
red = np.random.normal(0.12, 0.02, (size, size))
nir = np.random.normal(0.5, 0.05, (size, size))

# Stack into 4-band image
image = np.stack([blue, green, red, nir], axis=-1)

# -------------------------------------------
# 2. Inject Invasive Species Patch
# -------------------------------------------

# Define invasive region
image[80:130, 80:130, 3] += 0.15   # Increase NIR
image[80:130, 80:130, 2] -= 0.05   # Lower Red

# -------------------------------------------
# 3. Compute NDVI
# -------------------------------------------

ndvi = (nir - red) / (nir + red + 1e-6)

# -------------------------------------------
# 4. Prepare Data for Anomaly Detection
# -------------------------------------------

pixels = image.reshape(-1, 4)

# Add NDVI as additional feature
ndvi_flat = ndvi.reshape(-1, 1)
features = np.hstack([pixels, ndvi_flat])

# Isolation Forest
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(features)

pred = model.predict(features)
anomaly_map = pred.reshape(size, size)

# IsolationForest outputs:
# -1 = anomaly, 1 = normal
anomaly_map = np.where(anomaly_map == -1, 1, 0)

# -------------------------------------------
# 5. Visualization
# -------------------------------------------

plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.imshow(ndvi, cmap="Greens")
plt.title("NDVI")
plt.colorbar()

plt.subplot(1,3,2)
plt.imshow(anomaly_map, cmap="Reds")
plt.title("Detected Invasive Areas")

plt.subplot(1,3,3)
plt.imshow(anomaly_map, cmap="gray")
plt.title("Binary Invasive Mask")

plt.tight_layout()
plt.show()

# -------------------------------------------
# 6. Export Results
# -------------------------------------------

df_output = pd.DataFrame({
    "NDVI": ndvi_flat.flatten(),
    "Anomaly_Label": anomaly_map.flatten()
})

df_output.to_excel("invasive_species_detection_results.xlsx", index=False)

print("Invasive species detection results exported.")
